# MS-SQL Server Embeddings Generator

## Nuget Packages

In [1]:
#r "nuget: Microsoft.ML,3.0.1"
#r "nuget: Autofac"
#r "nuget: Microsoft.ML.Tokenizers"

Installed Packages Autofac, 8.0.0 Microsoft.ML, 3.0.1 Microsoft.ML.Tokenizers, 0.21.1

## Import Dependencies

In [16]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text.RegularExpressions;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.Text;

## Embeddings Generator Interface

The `IEmbeddingGenerator` interface has a single method called `GenerateEmbedding()`, which takes in a text string, a list of entities, a primary key, and a list of foreign keys as parameters. It returns a float array representing the embedding of the input text.

The purpose of this interface is to provide a common structure for different embedding generators to follow. By implementing this interface, classes can ensure that they have the necessary functionality to generate embeddings for text data.

In [3]:
// Interface for embedding generators
public interface IEmbeddingGenerator
{
        float[] GenerateEmbedding(string text, List<string> entities, string primaryKey, List<string> foreignKeys);
}

### EnhancedEmbeddingGenerator Class

The `EnhancedEmbeddingGenerator` class is designed to generate embeddings for text data, incorporating additional weights for primary keys, foreign keys, and entities.

#### Detailed Description:

1. **Constructor**:
   - **Parameters**:
     - `WordIndex wordIndex`: An instance of the `WordIndex` class, which manages the mapping of words to unique indices.
     - `int embeddingSize`: The size of the embedding vector.
   - **Initialization**: The constructor initializes the `wordIndex` and `embeddingSize` fields.

2. **GenerateEmbedding Method**:
   - **Parameters**:
     - `string text`: The input text to be embedded.
     - `List<string> entities`: A list of entities that should be given extra weight in the embedding.
     - `string primaryKey`: The primary key that should be given the highest weight.
     - `List<string> foreignKeys`: A list of foreign keys that should be given additional weight.
   - **Returns**: A `float[]` representing the embedding of the input text.
   - **Process**:
     - **Text Processing**: The input text is split into words. Each word is processed to update the embedding vector.
     - **Word Indexing**: Each word is mapped to an index using the `wordIndex` instance. The embedding vector is updated based on the index.
     - **Weight Adjustments**:
       - **Primary Key**: Words containing the primary key are given extra weight.
       - **Foreign Keys**: Words containing any of the foreign keys are given additional weight.
       - **Entities**: Entities are processed separately and given a significant weight.
     - **Normalization**: The embedding vector is normalized to unit length to ensure that the magnitude of the vector does not affect the downstream tasks.

This class provides a robust way to generate embeddings that take into account the importance of certain words (like primary keys and foreign keys) and entities, making the embeddings more meaningful for specific use cases. The normalization step ensures that the embeddings are on a comparable scale, which is crucial for many machine learning algorithms.

In [6]:
public class PrimaryKeyAwareEmbeddingGenerator : IEmbeddingGenerator
{
    private readonly WordIndex wordIndex;
    private readonly int embeddingSize;

    public PrimaryKeyAwareEmbeddingGenerator(WordIndex wordIndex, int embeddingSize)
    {
        this.wordIndex = wordIndex;
        this.embeddingSize = embeddingSize;
    }

    public float[] GenerateEmbedding(string text, List<string> entities, string primaryKey, List<string> foreignKeys)
    {
        var words = text.Split(' ');
        var embedding = new float[embeddingSize];

        // Process words
        for (int i = 0; i < words.Length; i++)
        {
            int wordIndex = this.wordIndex.GetOrAddWord(words[i]);
            embedding[wordIndex % embeddingSize] += 1;

            // Give substantial extra weight to the primary key
            if (words[i].Contains(primaryKey))
            {
                embedding[wordIndex % embeddingSize] += 10;
            }

            // Give some weight to foreign keys, but less than primary key
            if (foreignKeys.Any(fk => words[i].Contains(fk)))
            {
                embedding[wordIndex % embeddingSize] += 3;
            }
        }

        // Process entities
        foreach (var entity in entities)
        {
            int entityIndex = this.wordIndex.GetOrAddWord(entity);
            embedding[entityIndex % embeddingSize] += 2;
        }

        // Process primary key separately for additional emphasis
        if (!string.IsNullOrEmpty(primaryKey))
        {
            int primaryKeyIndex = this.wordIndex.GetOrAddWord(primaryKey);
            embedding[primaryKeyIndex % embeddingSize] += 15;

            // Also emphasize words often associated with primary keys
            var pkRelatedWords = new[] { "primary", "key", "id", "identifier" };
            foreach (var word in pkRelatedWords)
            {
                int relatedWordIndex = this.wordIndex.GetOrAddWord(word);
                embedding[relatedWordIndex % embeddingSize] += 5;
            }
        }

        // Process data types often used for primary keys
        var pkDataTypes = new[] { "int", "bigint", "uuid", "guid" };
        foreach (var dataType in pkDataTypes)
        {
            if (text.Contains(dataType))
            {
                int dataTypeIndex = this.wordIndex.GetOrAddWord(dataType);
                embedding[dataTypeIndex % embeddingSize] += 3;
            }
        }

        // Normalize the embedding
        float magnitude = (float)Math.Sqrt(embedding.Sum(x => x * x));
        if (magnitude > 0)
        {
            for (int i = 0; i < embedding.Length; i++)
            {
                embedding[i] /= magnitude;
            }
        }

        return embedding;
    }
}

### MLNetEmbeddingGenerator Class

The `MLNetEmbeddingGenerator` class is designed to generate embeddings for text data using ML.NET. This class implements the `IEmbeddingGenerator` interface and provides a method to generate embeddings that leverage ML.NET's text processing and feature extraction capabilities.

#### Detailed Description:

1. **Constructor**:
   - **Parameters**:
     - `int embeddingSize`: The size of the embedding vector, with a default value of 100.
   - **Initialization**:
     - Initializes an instance of `MLContext` with a seed for reproducibility.
     - Sets the embedding size.
     - Creates a text processing pipeline that includes normalization, tokenization, stop word removal, n-gram production, latent Dirichlet allocation (LDA) for topic modeling, and min-max normalization.
     - Fits the pipeline to a dummy dataset to create the model.

2. **GenerateEmbedding Method**:
   - **Parameters**:
     - `string text`: The input text to be embedded.
     - `List<string> entities`: A list of entities to be included in the embedding.
     - `string primaryKey`: The primary key to be included in the embedding.
     - `List<string> foreignKeys`: A list of foreign keys to be included in the embedding.
   - **Returns**: A `float[]` representing the embedding of the input text.
   - **Process**:
     - Combines the input text, entities, primary key, and foreign keys into a single string.
     - Creates a `DataView` with the combined text.
     - Uses the model to transform the data and generate embeddings.
     - Extracts the embedding from the transformed data.

3. **TextData Class**:
   - Represents the input text data with a single property `Text`.

4. **EmbeddingOutput Class**:
   - Represents the output embedding with a single property `Embedding`, which is a float array of the specified embedding size.

This class provides a robust way to generate embeddings using ML.NET's powerful text processing and feature extraction tools. The embeddings can be used for various machine learning tasks such as similarity search, clustering, and classification.

In [ ]:
public class MLNetEmbeddingGenerator : IEmbeddingGenerator
{
    private readonly MLContext _mlContext;
    private readonly ITransformer _model;
    private readonly int _embeddingSize;

    public MLNetEmbeddingGenerator(int embeddingSize = 100)
    {
        _mlContext = new MLContext(seed: 0);
        _embeddingSize = embeddingSize;

        // Create a simple pipeline for text featurization
        var pipeline = _mlContext.Transforms.Text.NormalizeText("NormalizedText", "Text")
            .Append(_mlContext.Transforms.Text.TokenizeIntoWords("Words", "NormalizedText"))
            .Append(_mlContext.Transforms.Text.RemoveDefaultStopWords("Words"))
            .Append(_mlContext.Transforms.Text.ProduceNgrams("NGrams", "Words"))
            .Append(_mlContext.Transforms.Text.LatentDirichletAllocation("Topics", "NGrams", numberOfTopics: _embeddingSize))
            .Append(_mlContext.Transforms.NormalizeMinMax("Embedding", "Topics"));

        // Fit the pipeline to create a model
        var dummyData = _mlContext.Data.LoadFromEnumerable(new List<TextData> { new TextData { Text = "Dummy text for model initialization" } });
        _model = pipeline.Fit(dummyData);
    }

    public float[] GenerateEmbedding(string text, List<string> entities, string primaryKey, List<string> foreignKeys)
    {
        // Combine all text fields into a single string
        string combinedText = $"{text} {string.Join(" ", entities)} {primaryKey} {string.Join(" ", foreignKeys)}";

        // Create a DataView with the input text
        var data = _mlContext.Data.LoadFromEnumerable(new List<TextData> { new TextData { Text = combinedText } });

        // Use the model to generate embeddings
        var transformedData = _model.Transform(data);

        // Extract the embedding from the transformed data
        var embeddings = _mlContext.Data.CreateEnumerable<EmbeddingOutput>(transformedData, reuseRowObject: false).ToList();
        return embeddings.FirstOrDefault()?.Embedding ?? new float[_embeddingSize];
    }

    private class TextData
    {
        [LoadColumn(0)]
        public string Text { get; set; }
    }

    private class EmbeddingOutput
    {
        [VectorType(_embeddingSize)]
        public float[] Embedding { get; set; }
    }
}

### EnhancedEmbeddingGenerator Class

The `EnhancedEmbeddingGenerator` class is designed to generate embeddings for text data, incorporating additional weights for primary keys, foreign keys, and entities. This class is part of a machine learning pipeline where embeddings are used to represent text in a numerical format suitable for various downstream tasks such as search, classification, or clustering.

#### Detailed Description:

1. **Constructor**:
   - **Parameters**:
     - `WordIndex wordIndex`: An instance of the `WordIndex` class, which manages the mapping of words to unique indices.
     - `int embeddingSize`: The size of the embedding vector.
   - **Initialization**: The constructor initializes the `wordIndex` and `embeddingSize` fields.

2. **GenerateEmbedding Method**:
   - **Parameters**:
     - `string text`: The input text to be embedded.
     - `List<string> entities`: A list of entities that should be given extra weight in the embedding.
     - `string primaryKey`: The primary key that should be given the highest weight.
     - `List<string> foreignKeys`: A list of foreign keys that should be given additional weight.
   - **Returns**: A `float[]` representing the embedding of the input text.
   - **Process**:
     - **Text Processing**: The input text is split into words. Each word is processed to update the embedding vector.
     - **Word Indexing**: Each word is mapped to an index using the `wordIndex` instance. The embedding vector is updated based on the index.
     - **Weight Adjustments**:
       - **Primary Key**: Words containing the primary key are given extra weight.
       - **Foreign Keys**: Words containing any of the foreign keys are given additional weight.
       - **Entities**: Entities are processed separately and given a significant weight.
     - **Normalization**: The embedding vector is normalized to unit length to ensure that the magnitude of the vector does not affect the downstream tasks.

This class provides a robust way to generate embeddings that take into account the importance of certain words (like primary keys and foreign keys) and entities, making the embeddings more meaningful for specific use cases. The normalization step ensures that the embeddings are on a comparable scale, which is crucial for many machine learning algorithms.

In [5]:
public class EnhancedEmbeddingGenerator : IEmbeddingGenerator
{
    private readonly WordIndex wordIndex;
    private readonly int embeddingSize;

    public EnhancedEmbeddingGenerator(WordIndex wordIndex, int embeddingSize)
    {
        this.wordIndex = wordIndex;
        this.embeddingSize = embeddingSize;
    }

    public float[] GenerateEmbedding(string text, List<string> entities, string primaryKey, List<string> foreignKeys)
    {
        var words = text.Split(' ');
        var embedding = new float[embeddingSize];

        // Process words
        for (int i = 0; i < words.Length; i++)
        {
            int wordIndex = this.wordIndex.GetOrAddWord(words[i]);
            embedding[wordIndex % embeddingSize] += 1;

            // Give extra weight to the primary key
            if (words[i].Contains(primaryKey))
            {
                embedding[wordIndex % embeddingSize] += 3;
            }

            // Give extra weight to foreign keys
            if (foreignKeys.Any(fk => words[i].Contains(fk)))
            {
                embedding[wordIndex % embeddingSize] += 2;
            }
        }

        // Process entities
        foreach (var entity in entities)
        {
            int entityIndex = this.wordIndex.GetOrAddWord(entity);
            embedding[entityIndex % embeddingSize] += 4;
        }

        // Process primary key separately
        int primaryKeyIndex = this.wordIndex.GetOrAddWord(primaryKey);
        embedding[primaryKeyIndex % embeddingSize] += 5;

        // Process foreign keys separately
        foreach (var foreignKey in foreignKeys)
        {
            int foreignKeyIndex = this.wordIndex.GetOrAddWord(foreignKey);
            embedding[foreignKeyIndex % embeddingSize] += 3;
        }

        // Normalize the embedding
        float magnitude = (float)Math.Sqrt(embedding.Sum(x => x * x));
        if (magnitude > 0)
        {
            for (int i = 0; i < embedding.Length; i++)
            {
                embedding[i] /= magnitude;
            }
        }

        return embedding;
    }
}

In [7]:
public class EmbeddingGenerator : IEmbeddingGenerator
{

    public float[] GenerateEmbedding(string text, List<string> entities, string primaryKey, List<string> foreignKeys)
    {
        return [];
    }
    
}

### ForeignKeyAwareEmbeddingGenerator Class

The `ForeignKeyAwareEmbeddingGenerator` class is designed to generate embeddings for text data, with a particular focus on foreign keys. This class implements the `IEmbeddingGenerator` interface and provides a method to generate embeddings that emphasize the importance of foreign keys, primary keys, and specific entities.

#### Detailed Description:

1. **Constructor**:
   - **Parameters**:
     - `WordIndex wordIndex`: An instance of the `WordIndex` class, which manages the mapping of words to unique indices.
     - `int embeddingSize`: The size of the embedding vector.
   - **Initialization**: The constructor initializes the `wordIndex` and `embeddingSize` fields.

2. **GenerateEmbedding Method**:
   - **Parameters**:
     - `string text`: The input text to be embedded.
     - `List<string> entities`: A list of entities that should be given extra weight in the embedding.
     - `string primaryKey`: The primary key that should be given a significant weight.
     - `List<string> foreignKeys`: A list of foreign keys that should be given substantial weight.
   - **Returns**: A `float[]` representing the embedding of the input text.
   - **Process**:
     - **Text Processing**: The input text is split into words. Each word is processed to update the embedding vector.
     - **Word Indexing**: Each word is mapped to an index using the `wordIndex` instance. The embedding vector is updated based on the index.
     - **Weight Adjustments**:
       - **Foreign Keys**: Words containing any of the foreign keys are given substantial extra weight.
       - **Primary Key**: Words containing the primary key are given some weight, but less than foreign keys.
       - **Entities**: Entities are processed separately and given additional weight.
       - **Foreign Key Emphasis**: Foreign keys are processed separately for additional emphasis, including related words often associated with foreign keys.
       - **Relationship Words**: Words related to relationships (e.g., "references", "delete", "cascade") are emphasized.
       - **Join Patterns**: Common join column patterns (e.g., "_id", "Id") are emphasized if they appear in foreign keys.
     - **Normalization**: The embedding vector is normalized to unit length to ensure that the magnitude of the vector does not affect downstream tasks.

This class provides a robust way to generate embeddings that take into account the importance of foreign keys, primary keys, and entities, making the embeddings more meaningful for specific use cases. The normalization step ensures that the embeddings are on a comparable scale, which is crucial for many machine learning algorithms.

In [8]:
public class ForeignKeyAwareEmbeddingGenerator : IEmbeddingGenerator
{
    private readonly WordIndex wordIndex;
    private readonly int embeddingSize;

    public ForeignKeyAwareEmbeddingGenerator(WordIndex wordIndex, int embeddingSize)
    {
        this.wordIndex = wordIndex;
        this.embeddingSize = embeddingSize;
    }

    public float[] GenerateEmbedding(string text, List<string> entities, string primaryKey, List<string> foreignKeys)
    {
        var words = text.Split(' ');
        var embedding = new float[embeddingSize];

        // Process words
        for (int i = 0; i < words.Length; i++)
        {
            int wordIndex = this.wordIndex.GetOrAddWord(words[i]);
            embedding[wordIndex % embeddingSize] += 1;

            // Give substantial extra weight to foreign keys
            if (foreignKeys.Any(fk => words[i].Contains(fk)))
            {
                embedding[wordIndex % embeddingSize] += 10;
            }

            // Give some weight to the primary key, but less than foreign keys
            if (words[i].Contains(primaryKey))
            {
                embedding[wordIndex % embeddingSize] += 3;
            }
        }

        // Process entities
        foreach (var entity in entities)
        {
            int entityIndex = this.wordIndex.GetOrAddWord(entity);
            embedding[entityIndex % embeddingSize] += 2;
        }

        // Process foreign keys separately for additional emphasis
        foreach (var foreignKey in foreignKeys)
        {
            int foreignKeyIndex = this.wordIndex.GetOrAddWord(foreignKey);
            embedding[foreignKeyIndex % embeddingSize] += 15;

            // Emphasize words often associated with foreign keys
            var fkRelatedWords = new[] { "foreign", "key", "reference", "constraint" };
            foreach (var word in fkRelatedWords)
            {
                int relatedWordIndex = this.wordIndex.GetOrAddWord(word);
                embedding[relatedWordIndex % embeddingSize] += 5;
            }
        }

        // Emphasize relationship-related words
        var relationshipWords = new[] { "references", "on", "delete", "cascade", "set", "null", "update" };
        foreach (var word in relationshipWords)
        {
            if (text.Contains(word))
            {
                int wordIndex = this.wordIndex.GetOrAddWord(word);
                embedding[wordIndex % embeddingSize] += 3;
            }
        }

        // Emphasize common join column patterns
        var joinPatterns = new[] { "_id", "Id", "_fk", "Fk" };
        foreach (var pattern in joinPatterns)
        {
            if (foreignKeys.Any(fk => fk.Contains(pattern)))
            {
                int patternIndex = this.wordIndex.GetOrAddWord(pattern);
                embedding[patternIndex % embeddingSize] += 4;
            }
        }

        // Normalize the embedding
        float magnitude = (float)Math.Sqrt(embedding.Sum(x => x * x));
        if (magnitude > 0)
        {
            for (int i = 0; i < embedding.Length; i++)
            {
                embedding[i] /= magnitude;
            }
        }

        return embedding;
    }
}

## WordIndex Class

The `WordIndex` class is designed to manage the mapping between words and their unique indices. This class is useful in natural language processing tasks where words need to be converted to numerical indices for further processing, such as in embedding generation or text classification.

#### Detailed Description:

1. **Private Fields**:
   - `Dictionary<string, int> wordToIndex`: A dictionary that maps words to their unique indices.
   - `List<string> indexToWord`: A list that maps indices back to their corresponding words.

2. **GetOrAddWord Method**:
   - **Parameters**:
     - `string word`: The word to be added or retrieved.
   - **Returns**: The index of the word.
   - **Functionality**: 
     - Checks if the word already exists in the `wordToIndex` dictionary.
     - If the word does not exist, it assigns a new index to the word, adds it to both the dictionary and the list, and returns the new index.
     - If the word exists, it simply returns the existing index.

3. **GetWord Method**:
   - **Parameters**:
     - `int index`: The index for which the corresponding word is to be retrieved.
   - **Returns**: The word corresponding to the given index.
   - **Functionality**: 
     - Checks if the index is within the valid range.
     - If the index is valid, it returns the word from the `indexToWord` list.
     - If the index is out of range, it throws an `ArgumentOutOfRangeException`.

4. **Count Property**:
   - **Returns**: The number of words currently stored in the `indexToWord` list.
   - **Functionality**: Provides the count of unique words managed by the `WordIndex` class.

This class provides a simple yet effective way to manage the mapping between words and their unique indices, which is a common requirement in many text processing and machine learning tasks.

In [4]:
public class WordIndex
{
    private Dictionary<string, int> wordToIndex = new Dictionary<string, int>();
    private List<string> indexToWord = new List<string>();

    public int GetOrAddWord(string word)
    {
        if (!wordToIndex.TryGetValue(word, out int index))
        {
            index = indexToWord.Count;
            wordToIndex[word] = index;
            indexToWord.Add(word);
        }
        return index;
    }

    public string GetWord(int index)
    {
        if (index < 0 || index >= indexToWord.Count)
        {
            throw new ArgumentOutOfRangeException(nameof(index), "Index is out of range.");
        }
        return indexToWord[index];
    }

    public int Count => indexToWord.Count;
}

In [9]:
var wordIndex = new WordIndex();
const int embeddingSize = 3072;

// Create instances of different embedding generators
var enhancedGenerator = new EnhancedEmbeddingGenerator(wordIndex, embeddingSize);
static var embeddingGenerator = new EmbeddingGenerator();
var primaryKeyAwareGenerator = new PrimaryKeyAwareEmbeddingGenerator(wordIndex, embeddingSize);
var foreignKeyAwareGenerator = new ForeignKeyAwareEmbeddingGenerator(wordIndex, embeddingSize);


### DocumentItem Class

The `DocumentItem` class is designed to represent a document item with various properties that are essential for text processing and embedding tasks. This class is particularly useful in scenarios where documents need to be indexed, categorized, and searched based on their content and metadata.

#### Detailed Description:

1. **Properties**:
   - **Id**: A unique identifier for the document.
   - **Text**: The text content of the document.
   - **Embedding**: A float array representing the embedding of the document's text. This embedding can be used for various machine learning tasks such as similarity search, clustering, and classification.
   - **SqlEntityReference**: A list of SQL entity references associated with the document. This can include table names, column names, or other SQL-related entities that are relevant to the document.
   - **Category**: A new property that represents the category of the document. This can be used to classify documents into different categories for better organization and retrieval.

This class provides a structured way to manage document items, making it easier to handle various text processing tasks and integrate with machine learning models. The inclusion of the `Category` property adds an additional layer of metadata that can be leveraged for more advanced document management and retrieval systems.

In [10]:
// Class to represent a document item
public class DocumentItem
{
    public string Id { get; set; }
    public string Text { get; set; }
    public float[] Embedding { get; set; }
    public List<string> SqlEntityReference { get; set; } = new List<string>();
    public string Category { get; set; }  // New property
}

In [11]:
static List<DocumentItem> sqlTestDocs = new List<DocumentItem>
    {
        new DocumentItem
        {
            Id = "loan_applications",
            Text = @"
CREATE TABLE LoanApplications (
    ApplicationID INT PRIMARY KEY,
    BorrowerID INT,
    ApplicationDate DATE,
    LoanAmount DECIMAL(15, 2),
    LoanPurpose VARCHAR(50),
    PropertyAddress VARCHAR(200),
    PropertyValue DECIMAL(15, 2),
    CreditScore INT,
    EmploymentStatus VARCHAR(50),
    AnnualIncome DECIMAL(15, 2),
    Status VARCHAR(20),
    FOREIGN KEY (BorrowerID) REFERENCES Borrowers(BorrowerID)
);",
            SqlEntityReference = new List<string>{"Mortgage", "Loan Origination", "Applications", "BorrowerID", "LoanAmount"}
        },
        new DocumentItem
        {
            Id = "borrowers",
            Text = @"
CREATE TABLE Borrowers (
    BorrowerID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    DateOfBirth DATE,
    SSN VARCHAR(11),
    Email VARCHAR(100),
    PhoneNumber VARCHAR(20),
    Address VARCHAR(200),
    City VARCHAR(50),
    State VARCHAR(2),
    ZipCode VARCHAR(10)
);",
            SqlEntityReference = new List<string>{"Mortgage", "Loan Origination", "Borrowers", "BorrowerID", "SSN"}
        },
        new DocumentItem
        {
            Id = "loans",
            Text = @"
CREATE TABLE Loans (
    LoanID INT PRIMARY KEY,
    ApplicationID INT,
    BorrowerID INT,
    OriginationDate DATE,
    LoanAmount DECIMAL(15, 2),
    InterestRate DECIMAL(5, 3),
    LoanTerm INT,
    MonthlyPayment DECIMAL(10, 2),
    PropertyAddress VARCHAR(200),
    LoanType VARCHAR(50),
    LoanStatus VARCHAR(20),
    FOREIGN KEY (ApplicationID) REFERENCES LoanApplications(ApplicationID),
    FOREIGN KEY (BorrowerID) REFERENCES Borrowers(BorrowerID)
);",
            SqlEntityReference = new List<string>{"Mortgage", "Loan Servicing", "Loans", "LoanID", "ApplicationID", "BorrowerID"}
        },
        new DocumentItem
        {
            Id = "loan_payments",
            Text = @"
CREATE TABLE LoanPayments (
    PaymentID INT PRIMARY KEY,
    LoanID INT,
    PaymentDate DATE,
    PaymentAmount DECIMAL(10, 2),
    PrincipalAmount DECIMAL(10, 2),
    InterestAmount DECIMAL(10, 2),
    LateFee DECIMAL(7, 2),
    FOREIGN KEY (LoanID) REFERENCES Loans(LoanID)
);",
            SqlEntityReference = new List<string>{"Mortgage", "Loan Servicing", "Payments", "PaymentID", "LoanID"}
        },
        new DocumentItem
        {
            Id = "loan_sales",
            Text = @"
CREATE TABLE LoanSales (
    SaleID INT PRIMARY KEY,
    LoanID INT,
    BuyerID INT,
    SaleDate DATE,
    SaleAmount DECIMAL(15, 2),
    SaleType VARCHAR(50),
    FOREIGN KEY (LoanID) REFERENCES Loans(LoanID),
    FOREIGN KEY (BuyerID) REFERENCES LoanBuyers(BuyerID)
);",
            SqlEntityReference = new List<string>{"Mortgage", "Loan Sales", "Sales", "SaleID", "LoanID", "BuyerID"}
        },
        new DocumentItem
        {
            Id = "loan_buyers",
            Text = @"
CREATE TABLE LoanBuyers (
    BuyerID INT PRIMARY KEY,
    BuyerName VARCHAR(100),
    BuyerType VARCHAR(50),
    ContactPerson VARCHAR(100),
    Email VARCHAR(100),
    PhoneNumber VARCHAR(20),
    Address VARCHAR(200),
    City VARCHAR(50),
    State VARCHAR(2),
    ZipCode VARCHAR(10)
);",
            SqlEntityReference = new List<string>{"Mortgage", "Loan Sales", "Buyers", "BuyerID"}
        },
        new DocumentItem
        {
            Id = "property_appraisals",
            Text = @"
            CREATE TABLE PropertyAppraisals (
                AppraisalID INT PRIMARY KEY,
                LoanID INT,
                AppraisalDate DATE,
                AppraisedValue DECIMAL(15, 2),
                AppraiserName VARCHAR(100),
                AppraisalMethod VARCHAR(50),
                PropertyCondition VARCHAR(50),
                FOREIGN KEY (LoanID) REFERENCES Loans(LoanID)
            );",
                        SqlEntityReference = new List<string>{"Mortgage", "Property", "Appraisals", "AppraisalID", "LoanID"}
                    },
                    new DocumentItem
                    {
                        Id = "loan_underwriting",
                        Text = @"
            CREATE TABLE LoanUnderwriting (
                UnderwritingID INT PRIMARY KEY,
                LoanID INT,
                UnderwriterID INT,
                UnderwritingDate DATE,
                Decision VARCHAR(20),
                RiskScore INT,
                Notes TEXT,
                FOREIGN KEY (LoanID) REFERENCES Loans(LoanID),
                FOREIGN KEY (UnderwriterID) REFERENCES Employees(EmployeeID)
            );",
                        SqlEntityReference = new List<string>{"Mortgage", "Loan Origination", "Underwriting", "UnderwritingID", "LoanID"}
                    },
                    new DocumentItem
                    {
                        Id = "employees",
                        Text = @"
            CREATE TABLE Employees (
                EmployeeID INT PRIMARY KEY,
                FirstName VARCHAR(50),
                LastName VARCHAR(50),
                Department VARCHAR(50),
                Position VARCHAR(50),
                Email VARCHAR(100),
                PhoneNumber VARCHAR(20),
                HireDate DATE,
                ManagerID INT,
                FOREIGN KEY (ManagerID) REFERENCES Employees(EmployeeID)
            );",
                        SqlEntityReference = new List<string>{"Mortgage", "HR", "Employees", "EmployeeID"}
                    }
                };

### PreprocessedDocument Class

The `PreprocessedDocument` class represents a document that has been preprocessed for further analysis. This class includes various properties that capture the essential elements of the document, such as its text, primary key, word indices, entities, and embedding.

#### Detailed Description:

- **Id**: A unique identifier for the document.
- **OriginalText**: The original text content of the document.
- **PrimaryKey**: The primary key associated with the document.
- **WordIndices**: A list of word indices representing the words in the document.
- **Entities**: A list of entities mentioned in the document.
- **PrimaryKeys**: A list of primary keys found in the document.
- **ForeignKeys**: A list of foreign keys found in the document.
- **Embedding**: A float array representing the embedding of the document's text.

### MultiModelSemanticSearchEngine Class

The `MultiModelSemanticSearchEngine` class is the main class for the semantic search engine. It preprocesses documents, generates embeddings using multiple models, and performs searches based on query embeddings and entity overlap.

#### Detailed Description:

- **Constructor**:
  - **Parameters**:
    - `List<DocumentItem> rawDocuments`: The raw documents to be processed.
    - `List<(IEmbeddingGenerator generator, float weight)> embeddingGenerators`: A list of embedding generators and their respective weights.
    - `WordIndex wordIndex`: An instance of the `WordIndex` class.
    - `float entityWeight`: The weight assigned to entities in the relevance calculation.
  - **Initialization**: Initializes the embedding generators, word index, entity weight, and preprocesses the documents.

- **PreprocessDocuments Method**:
  - **Parameters**: `List<DocumentItem> rawDocuments`
  - **Returns**: A list of preprocessed documents.
  - **Functionality**: Extracts primary keys, foreign keys, generates embeddings, and assigns categories to the documents.

- **GenerateCombinedEmbedding Method**:
  - **Parameters**: `string text`, `List<string> entities`, `string primaryKey`, `List<string> foreignKeys`
  - **Returns**: A combined embedding for the input text.
  - **Functionality**: Generates embeddings using multiple models and combines them with respective weights.

- **CalculateCategoryScore Method**:
  - **Parameters**: `List<string> queryEntities`, `string docCategory`
  - **Returns**: A score based on the overlap between query entities and document category.

- **CalculateEntityOverlap Method**:
  - **Parameters**: `List<string> queryEntities`, `List<string> docEntities`
  - **Returns**: A score based on the overlap between query entities and document entities.

- **CalculateRelevanceScore Method**:
  - **Parameters**: `float[] queryEmbedding`, `Document doc`, `List<string> queryEntities`, `bool isForeignKeyQuery`, `string targetTable`
  - **Returns**: A relevance score for the document based on various factors.

- **CalculateForeignKeyScore Method**:
  - **Parameters**: `Document doc`, `string targetTable`
  - **Returns**: A score based on the presence of foreign keys related to the target table.

- **ExtractTargetTableFromQuery Method**:
  - **Parameters**: `string query`
  - **Returns**: The target table extracted from the query.

- **Search Method**:
  - **Parameters**: `string query`, `List<string> queryEntities`
  - **Returns**: A list of documents that match the query.
  - **Functionality**: Generates a query embedding, calculates relevance scores, and returns the top matching documents.

- **PreprocessText Method**:
  - **Parameters**: `string text`
  - **Returns**: The preprocessed text.
  - **Functionality**: Converts text to lowercase, removes non-alphanumeric characters, and trims whitespace.

- **CosineSimilarity Method**:
  - **Parameters**: `float[] v1`, `float[] v2`
  - **Returns**: The cosine similarity between two vectors.

- **GetCategory Method**:
  - **Parameters**: `List<string> entities`
  - **Returns**: The category of the document based on its entities.

- **CalculateCrossCategoryBonus Method**:
  - **Parameters**: `Dictionary<string, int> categoryCounts`, `string category`
  - **Returns**: A bonus score for cross-category relevance.

- **EvaluateMAP Method**:
  - **Parameters**: `List<(string query, List<string> queryEntities, List<string> relevantDocIds)> testSet`
  - **Returns**: The mean average precision (MAP) score for the test set.

- **CalculateAveragePrecision Method**:
  - **Parameters**: `List<string> rankedList`, `List<string> relevantDocIds`
  - **Returns**: The average precision for the ranked list of documents.

- **ExtractPrimaryKey Method**:
  - **Parameters**: `string text`
  - **Returns**: The primary key extracted from the text.

- **ExtractForeignKeys Method**:
  - **Parameters**: `string text`
  - **Returns**: A list of foreign keys extracted from the text.

### Program Class

The `Program` class contains the `Main` method, which serves as the entry point for the application. It initializes the necessary components, loads documents, creates the search engine, performs searches, and evaluates the search engine's performance.

#### Detailed Description:

- **Main Method**:
  - **Functionality**: Initializes the word index, embedding generators, and search engine. Loads documents, performs searches, and evaluates the search engine using mean average precision (MAP).

- **DisplayResults Method**:
  - **Parameters**: `List<(string Id, string Text, List<string> Entities, string Category)> results`
  - **Functionality**: Displays the search results in a readable format.

- **GetCategory Method**:
  - **Parameters**: `List<string> entities`
  - **Returns**: The category of the document based on its entities.

- **LoadMortgageSqlDocuments Method**:
  - **Returns**: A list of `DocumentItem` objects representing the mortgage SQL documents.
  - **Functionality**: Loads and assigns categories to the SQL documents.

In [13]:
// Class to represent a preprocessed document
public class PreprocessedDocument
{
    public string Id { get; set; }
    public string OriginalText { get; set; }
    public string PrimaryKey { get; set; }
    public List<int> WordIndices { get; set; }
    public List<string> Entities { get; set; }
    public List<string> PrimaryKeys { get; set; }
    public List<string> ForeignKeys { get; set; }
    public float[] Embedding { get; set; }
}


// Main class for the semantic search engine
public class MultiModelSemanticSearchEngine
{
    private readonly List<(string Id, string Text, List<string> Entities, string Category, float[] Embedding, List<string> ForeignKeys)> documents;
    private readonly List<(IEmbeddingGenerator generator, float weight)> embeddingGenerators;
    private readonly WordIndex wordIndex;
    private readonly float entityWeight;

    public MultiModelSemanticSearchEngine(
        List<DocumentItem> rawDocuments,
        List<(IEmbeddingGenerator generator, float weight)> embeddingGenerators,
        WordIndex wordIndex,
        float entityWeight = 0.3f)
    {
        this.embeddingGenerators = embeddingGenerators;
        this.wordIndex = wordIndex;
        this.entityWeight = entityWeight;
        this.documents = PreprocessDocuments(rawDocuments);
    }

    private List<(string Id, string Text, List<string> Entities, string Category, float[] Embedding, List<string> ForeignKeys)> PreprocessDocuments(List<DocumentItem> rawDocuments)
    {
        return rawDocuments.Select(doc =>
        {
            string primaryKey = ExtractPrimaryKey(doc.Text);
            List<string> foreignKeys = ExtractForeignKeys(doc.Text);
            float[] embedding = GenerateCombinedEmbedding(doc.Text, doc.SqlEntityReference, primaryKey, foreignKeys);
            string category = GetCategory(doc.SqlEntityReference);
            return (doc.Id, doc.Text, doc.SqlEntityReference, doc.Category, embedding, foreignKeys);
        }).ToList();
    }

    private float[] GenerateCombinedEmbedding(string text, List<string> entities, string primaryKey, List<string> foreignKeys)
    {
        var combinedEmbedding = new float[embeddingGenerators[0].generator.GenerateEmbedding(text, entities, primaryKey, foreignKeys).Length];

        foreach (var (generator, weight) in embeddingGenerators)
        {
            var embedding = generator.GenerateEmbedding(text, entities, primaryKey, foreignKeys);
            for (int i = 0; i < combinedEmbedding.Length; i++)
            {
                combinedEmbedding[i] += embedding[i] * weight;
            }
        }

        // Normalize the combined embedding
        float magnitude = (float)Math.Sqrt(combinedEmbedding.Sum(x => x * x));
        for (int i = 0; i < combinedEmbedding.Length; i++)
        {
            combinedEmbedding[i] /= magnitude;
        }

        return combinedEmbedding;
    }

    private float CalculateCategoryScore(List<string> queryEntities, string docCategory)
    {
        // Check if any query entity contains the document category
        return queryEntities.Any(e => e.Contains(docCategory)) ? 1.0f : 0.0f;
    }

    private float CalculateEntityOverlap(List<string> queryEntities, List<string> docEntities)
    {
        var commonEntities = queryEntities.Intersect(docEntities, StringComparer.OrdinalIgnoreCase).Count();
        return (float)commonEntities / Math.Max(queryEntities.Count, docEntities.Count);
    }


    private float CalculateRelevanceScore(float[] queryEmbedding, (string Id, string Text, List<string> Entities, string Category, float[] Embedding, List<string> ForeignKeys) doc, List<string> queryEntities, bool isForeignKeyQuery, string targetTable)
    {
        float similarityScore = CosineSimilarity(queryEmbedding, doc.Embedding);
        float entityScore = CalculateEntityOverlap(queryEntities, doc.Entities);
        float categoryScore = CalculateCategoryScore(queryEntities, doc.Category);
        float foreignKeyScore = 0;

        if (isForeignKeyQuery)
        {
            foreignKeyScore = CalculateForeignKeyScore(doc, targetTable);
        }

        // Adjust weights based on query type and scores
        float textWeight = isForeignKeyQuery ? 0.2f : 0.4f;
        float entityWeight = isForeignKeyQuery ? 0.2f : 0.3f;
        float categoryWeight = 0.3f;
        float foreignKeyWeight = isForeignKeyQuery ? 0.3f : 0f;

        // Apply a relevance threshold
        float combinedScore = (textWeight * similarityScore) + 
                              (entityWeight * entityScore) + 
                              (categoryWeight * categoryScore) + 
                              (foreignKeyWeight * foreignKeyScore);

        // Return 0 if the combined score is below a threshold
        return combinedScore > 0.4f ? combinedScore : 0f;
    }

    private float CalculateForeignKeyScore((string Id, string Text, List<string> Entities, string Category, float[] Embedding, List<string> ForeignKeys) doc, string targetTable)
    {
        if (doc.Id.Equals(targetTable, StringComparison.OrdinalIgnoreCase))
        {
            return 0; // Penalize the target table itself
        }

        bool hasForeignKeyToTarget = doc.ForeignKeys.Any(fk => fk.Equals(targetTable, StringComparison.OrdinalIgnoreCase));
        return hasForeignKeyToTarget ? 1 : 0;
    }

    private string ExtractTargetTableFromQuery(string query)
    {
        var match = Regex.Match(query, @"to\s+(\w+)\s+table", RegexOptions.IgnoreCase);
        return match.Success ? match.Groups[1].Value : string.Empty;
    }

    public List<(string Id, string Text, List<string> Entities, string Category)> Search(string query, List<string> queryEntities)
    {
        var queryEmbedding = GenerateCombinedEmbedding(query, queryEntities, "", new List<string>());

        bool isForeignKeyQuery = query.ToLower().Contains("foreign key");
        string targetTable = ExtractTargetTableFromQuery(query);

        var results = documents
            .Select(doc => new
            {
                Document = doc,
                RelevanceScore = CalculateRelevanceScore(queryEmbedding, doc, queryEntities, isForeignKeyQuery, targetTable)
            })
            .Where(x => x.RelevanceScore > 0) // Filter out irrelevant results
            .OrderByDescending(x => x.RelevanceScore)
            .Take(5)
            .Select(x => (x.Document.Id, x.Document.Text, x.Document.Entities, x.Document.Category))
            .ToList();

        return results;
    }

    private string PreprocessText(string text)
    {
        text = text.ToLowerInvariant();
        text = Regex.Replace(text, @"[^\w\s]", "");
        text = Regex.Replace(text, @"\s+", " ").Trim();
        return text;
    }

    private float CosineSimilarity(float[] v1, float[] v2)
    {
        float dotProduct = 0;
        float magnitude1 = 0;
        float magnitude2 = 0;

        for (int i = 0; i < v1.Length; i++)
        {
            dotProduct += v1[i] * v2[i];
            magnitude1 += v1[i] * v1[i];
            magnitude2 += v2[i] * v2[i];
        }

        magnitude1 = (float)Math.Sqrt(magnitude1);
        magnitude2 = (float)Math.Sqrt(magnitude2);

        if (magnitude1 > 0 && magnitude2 > 0)
        {
            return dotProduct / (magnitude1 * magnitude2);
        }
        else
        {
            return 0;
        }
    }

    static string GetCategory(List<string> entities)
    {
        if (entities.Any(e => e.Contains("Loan Origination"))) return "Loan Origination";
        if (entities.Any(e => e.Contains("Loan Servicing"))) return "Loan Servicing";
        if (entities.Any(e => e.Contains("Loan Sales"))) return "Loan Sales";
        if (entities.Any(e => e.Contains("Property"))) return "Property";
        if (entities.Any(e => e.Contains("HR"))) return "HR";
        return "Other";
    }

    private float CalculateCrossCategoryBonus(Dictionary<string, int> categoryCounts, string category)
    {
        float totalCount = categoryCounts.Values.Sum();
        float categoryRatio = categoryCounts[category] / totalCount;
        return (1 - categoryRatio) * 0.1f;  // Small bonus for cross-category relevance
    }

    public float EvaluateMAP(List<(string query, List<string> queryEntities, List<string> relevantDocIds)> testSet)
    {
        float totalAP = 0;
        foreach (var (query, queryEntities, relevantDocIds) in testSet)
        {
            var results = Search(query, queryEntities);
            float ap = CalculateAveragePrecision(results.Select(r => r.Id).ToList(), relevantDocIds);
            totalAP += ap;
        }
        return totalAP / testSet.Count;
    }

    private float CalculateAveragePrecision(List<string> rankedList, List<string> relevantDocIds)
    {
        float relevantCount = 0;
        float totalPrecision = 0;

        for (int i = 0; i < rankedList.Count; i++)
        {
            if (relevantDocIds.Contains(rankedList[i]))
            {
                relevantCount++;
                float precision = relevantCount / (i + 1);
                totalPrecision += precision;
            }
        }

        return relevantCount > 0 ? totalPrecision / relevantCount : 0;
    }

    private string ExtractPrimaryKey(string text)
    {
        // Simple extraction of primary key - you might want to improve this
        var match = Regex.Match(text, @"(\w+)\s+(?:INT|VARCHAR\(\d+\))\s+PRIMARY KEY");
        return match.Success ? match.Groups[1].Value : string.Empty;
    }

    private List<string> ExtractForeignKeys(string text)
    {
        // Simple extraction of foreign keys - you might want to improve this
        var matches = Regex.Matches(text, @"FOREIGN KEY\s*\((\w+)\)");
        return matches.Cast<Match>().Select(m => m.Groups[1].Value).ToList();
    }
}

class Program
{
    public 
static void Main(string[] args)
    {
        var wordIndex = new WordIndex();
        const int embeddingSize = 3072;

        var enhancedGenerator = new EnhancedEmbeddingGenerator(wordIndex, embeddingSize);
        var primaryKeyAwareGenerator = new PrimaryKeyAwareEmbeddingGenerator(wordIndex, embeddingSize);
        var foreignKeyAwareGenerator = new ForeignKeyAwareEmbeddingGenerator(wordIndex, embeddingSize);

        var embeddingGenerators = new List<(IEmbeddingGenerator generator, float weight)>
        {
            (enhancedGenerator, 0.4f),
            (primaryKeyAwareGenerator, 0.3f),
            (foreignKeyAwareGenerator, 0.3f)
        };

        // Load documents (assuming we have a method to load the SQL documents)
        List<DocumentItem> mortgageSqlDocuments = LoadMortgageSqlDocuments();

        // Create search engine
        var multiModelSearchEngine = new MultiModelSemanticSearchEngine(
            mortgageSqlDocuments,
            embeddingGenerators,
            wordIndex,
            entityWeight: 0.3f
        );

        // Perform searches
        var testQueries = new List<(string query, List<string> queryEntities)>
        {
            ("Find all loan origination related tables", new List<string> { "Mortgage", "Loan Origination" }),
            ("Show tables with borrower information", new List<string> { "Mortgage", "Borrowers" }),
            ("List all loan servicing tables", new List<string> { "Mortgage", "Loan Servicing" }),
            ("Find tables related to loan sales", new List<string> { "Mortgage", "Loan Sales" }),
            ("Show tables with property information", new List<string> { "Mortgage", "Property" }),
            ("Find tables with payment information", new List<string> { "Mortgage", "Payments" }),
            ("List tables related to underwriting process", new List<string> { "Mortgage", "Underwriting" })
        };

        foreach (var (query, queryEntities) in testQueries)
        {
            Console.WriteLine($"\nQuery: {query}");
            Console.WriteLine($"Entities: {string.Join(", ", queryEntities)}");
            Console.WriteLine("Search results:");
            var results = multiModelSearchEngine.Search(query, queryEntities);
            DisplayResults(results);
            Console.WriteLine("----------------------------------------");
        }

        // Evaluate MAP
        var testSet = new List<(string query, List<string> queryEntities, List<string> relevantDocIds)>
        {
            ("Find loan origination tables", new List<string> { "Mortgage", "Loan Origination" }, new List<string> { "loan_applications", "borrowers", "loan_underwriting" }),
            ("Show borrower information", new List<string> { "Mortgage", "Borrowers" }, new List<string> { "borrowers", "loan_applications" }),
            ("List loan servicing tables", new List<string> { "Mortgage", "Loan Servicing" }, new List<string> { "loans", "loan_payments" }),
            ("Find loan sales related tables", new List<string> { "Mortgage", "Loan Sales" }, new List<string> { "loan_sales", "loan_buyers" }),
            ("Show property appraisal information", new List<string> { "Mortgage", "Property" }, new List<string> { "property_appraisals" })
        };

        float map = multiModelSearchEngine.EvaluateMAP(testSet);
        Console.WriteLine($"Mean Average Precision (MAP): {map}");
    }

    static void DisplayResults(List<(string Id, string Text, List<string> Entities, string Category)> results)
    {
        foreach (var (id, text, entities, category) in results)
        {
            Console.WriteLine($"ID: {id}");
            Console.WriteLine($"Category: {category}");
            Console.WriteLine($"Text: {text.Substring(0, Math.Min(100, text.Length))}..."); // Display first 100 characters
            Console.WriteLine($"Entities: {string.Join(", ", entities)}");
            Console.WriteLine();
        }
    }

    static string GetCategory(List<string> entities)
    {
        if (entities.Any(e => e.Contains("Loan Origination"))) return "Loan Origination";
        if (entities.Any(e => e.Contains("Loan Servicing"))) return "Loan Servicing";
        if (entities.Any(e => e.Contains("Loan Sales"))) return "Loan Sales";
        if (entities.Any(e => e.Contains("Property"))) return "Property";
        if (entities.Any(e => e.Contains("HR"))) return "HR";
        return "Other";
    }

    static List<DocumentItem> LoadMortgageSqlDocuments()
    {
        
        // Assign categories based on entities
        foreach (var doc in sqlTestDocs)
        {
            doc.Category = GetCategory(doc.SqlEntityReference);
        }

        return sqlTestDocs;
    }
}

### PrimaryKeyAwareEmbeddingGenerator Class

The `PrimaryKeyAwareEmbeddingGenerator` class is designed to generate embeddings for text data with a specific focus on primary keys. This class implements the `IEmbeddingGenerator` interface and provides a method to generate embeddings that emphasize the importance of primary keys and entities.

#### Detailed Description:

1. **Constructor**:
   - **Parameters**:
     - `WordIndex wordIndex`: An instance of the `WordIndex` class, which manages the mapping of words to unique indices.
     - `int embeddingSize`: The size of the embedding vector.
   - **Initialization**: The constructor initializes the `wordIndex` and `embeddingSize` fields.

2. **GenerateEmbedding Method**:
   - **Parameters**:
     - `string text`: The input text to be embedded.
     - `List<string> entities`: A list of entities that should be given extra weight in the embedding.
     - `string primaryKey`: The primary key that should be given significant weight.
     - `List<string> foreignKeys`: A list of foreign keys (not specifically used in this implementation).
   - **Returns**: A `float[]` representing the embedding of the input text.
   - **Process**:
     - **Text Processing**: The input text is split into words. Each word is processed to update the embedding vector.
     - **Word Indexing**: Each word is mapped to an index using the `wordIndex` instance. The embedding vector is updated based on the index.
     - **Weight Adjustments**:
       - **Primary Key**: Words containing the primary key are given substantial extra weight.
       - **Entities**: Entities are processed separately and given additional weight.
     - **Normalization**: The embedding vector is normalized to unit length to ensure that the magnitude of the vector does not affect downstream tasks.

This class provides a robust way to generate embeddings that take into account the importance of primary keys and entities, making the embeddings more meaningful for specific use cases. The normalization step ensures that the embeddings are on a comparable scale, which is crucial for many machine learning algorithms.

In [12]:
// New embedding generator considering Primary Key
public class PrimaryKeyAwareEmbeddingGenerator : IEmbeddingGenerator
{
    private readonly WordIndex wordIndex;
    private readonly int embeddingSize;

    public PrimaryKeyAwareEmbeddingGenerator(WordIndex wordIndex, int embeddingSize)
    {
        this.wordIndex = wordIndex;
        this.embeddingSize = embeddingSize;
    }

    public float[] GenerateEmbedding(string text, List<string> entities, string primaryKey, List<string> foreignKeys)
    {
        var words = text.Split(' ');
        var embedding = new float[embeddingSize];

        for (int i = 0; i < words.Length; i++)
        {
            int wordIndex = this.wordIndex.GetOrAddWord(words[i]);
            embedding[wordIndex % embeddingSize] += 1;

            // Give extra weight to the primary key
            if (words[i].Contains(primaryKey))
            {
                embedding[wordIndex % embeddingSize] += 5;
            }
        }

        foreach (var entity in entities)
        {
            int entityIndex = this.wordIndex.GetOrAddWord(entity);
            embedding[entityIndex % embeddingSize] += 3;
        }

        // Normalize the embedding
        float magnitude = (float)Math.Sqrt(embedding.Sum(x => x * x));
        for (int i = 0; i < embedding.Length; i++)
        {
            embedding[i] /= magnitude;
        }

        return embedding;
    }
}

### ForeignKeyAwareEmbeddingGenerator Class

The `ForeignKeyAwareEmbeddingGenerator` class is designed to generate embeddings for text data with a specific focus on foreign keys. This class implements the `IEmbeddingGenerator` interface and provides a method to generate embeddings that emphasize the importance of foreign keys and entities.

#### Detailed Description:

1. **Constructor**:
   - **Parameters**:
     - `WordIndex wordIndex`: An instance of the `WordIndex` class, which manages the mapping of words to unique indices.
     - `int embeddingSize`: The size of the embedding vector.
   - **Initialization**: The constructor initializes the `wordIndex` and `embeddingSize` fields.

2. **GenerateEmbedding Method**:
   - **Parameters**:
     - `string text`: The input text to be embedded.
     - `List<string> entities`: A list of entities that should be given extra weight in the embedding.
     - `string primaryKey`: The primary key (not specifically used in this implementation).
     - `List<string> foreignKeys`: A list of foreign keys that should be given significant weight.
   - **Returns**: A `float[]` representing the embedding of the input text.
   - **Process**:
     - **Text Processing**: The input text is split into words. Each word is processed to update the embedding vector.
     - **Word Indexing**: Each word is mapped to an index using the `wordIndex` instance. The embedding vector is updated based on the index.
     - **Weight Adjustments**:
       - **Foreign Keys**: Words containing any of the foreign keys are given extra weight.
       - **Entities**: Entities are processed separately and given additional weight.
     - **Normalization**: The embedding vector is normalized to unit length to ensure that the magnitude of the vector does not affect downstream tasks.

This class provides a robust way to generate embeddings that take into account the importance of foreign keys and entities, making the embeddings more meaningful for specific use cases. The normalization step ensures that the embeddings are on a comparable scale, which is crucial for many machine learning algorithms.

In [13]:
// New embedding generator considering Foreign Keys
public class ForeignKeyAwareEmbeddingGenerator : IEmbeddingGenerator
{
    private readonly WordIndex wordIndex;
    private readonly int embeddingSize;

    public ForeignKeyAwareEmbeddingGenerator(WordIndex wordIndex, int embeddingSize)
    {
        this.wordIndex = wordIndex;
        this.embeddingSize = embeddingSize;
    }

    public float[] GenerateEmbedding(string text, List<string> entities, string primaryKey, List<string> foreignKeys)
    {
        var words = text.Split(' ');
        var embedding = new float[embeddingSize];

        for (int i = 0; i < words.Length; i++)
        {
            int wordIndex = this.wordIndex.GetOrAddWord(words[i]);
            embedding[wordIndex % embeddingSize] += 1;

            // Give extra weight to foreign keys
            if (foreignKeys.Any(fk => words[i].Contains(fk)))
            {
                embedding[wordIndex % embeddingSize] += 3;
            }
        }

        foreach (var entity in entities)
        {
            int entityIndex = this.wordIndex.GetOrAddWord(entity);
            embedding[entityIndex % embeddingSize] += 2;
        }

        // Normalize the embedding
        float magnitude = (float)Math.Sqrt(embedding.Sum(x => x * x));
        for (int i = 0; i < embedding.Length; i++)
        {
            embedding[i] /= magnitude;
        }

        return embedding;
    }
}

In [15]:
Program.Main(new string[0]);


Query: Find all loan origination related tables
Entities: Mortgage, Loan Origination
Search results:
ID: borrowers
Category: Loan Origination
Text: 
CREATE TABLE Borrowers (
    BorrowerID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VA...
Entities: Mortgage, Loan Origination, Borrowers, BorrowerID, SSN

ID: loan_underwriting
Category: Loan Origination
Text: 
            CREATE TABLE LoanUnderwriting (
                UnderwritingID INT PRIMARY KEY,
       ...
Entities: Mortgage, Loan Origination, Underwriting, UnderwritingID, LoanID

ID: loan_applications
Category: Loan Origination
Text: 
CREATE TABLE LoanApplications (
    ApplicationID INT PRIMARY KEY,
    BorrowerID INT,
    Applicat...
Entities: Mortgage, Loan Origination, Applications, BorrowerID, LoanAmount

----------------------------------------

Query: Show tables with borrower information
Entities: Mortgage, Borrowers
Search results:
----------------------------------------

Query: List all loan servicing table